### Program on Machine Translation from English to Hindi

In [1]:
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model
import pandas as pd
import numpy as np
import string
from string import digits
import re
import matplotlib.pyplot as plt
import keras

C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Reading the data and creating dataframe with English sentences and corresponding hinid sentences

In [28]:
eword=[]
hword=[]
with open('C:\\Users\\richa\\Desktop\\conversica\\data.txt',encoding='utf-8') as f:
    for line in f:
        line=line.split('\t')
        eword.append(line[0])
        hword.append(line[1])

lines =pd.DataFrame([eword,hword]).T
lines.columns =['eword','hword']

In [29]:
lines['hword']=lines['hword'].apply(lambda x : x.rstrip())

In [30]:
### I considered only 30000 lines as going for more lines gave memory error on my machine

In [31]:
lines =lines[0:30000]

In [32]:
lines.shape

(30000, 2)

In [33]:
lines.head(2)

,eword,hword
0,Go.,चले जाओ!
1,Run!,रन !


### Data Pre-processing

In [34]:
lines.eword = lines.eword.apply(lambda x: x.lower())
lines.hword =lines.hword.apply(lambda x: x.lower())

In [35]:
lines.eword=lines.eword.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))
lines.hword=lines.hword.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))

In [36]:
exclude = set(string.punctuation)
lines.eword=lines.eword.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.hword=lines.hword.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [37]:
remove_digits = str.maketrans('', '', digits)
lines.eword=lines.eword.apply(lambda x: x.translate(remove_digits))
lines.hword=lines.hword.apply(lambda x: x.translate(remove_digits))

In [38]:
lines.hword = lines.hword.apply(lambda x : 'START_ '+ x + ' _END')

In [39]:
lines.head(2)

,eword,hword
0,go,START_ चले जाओ _END
1,run,START_ रन _END


In [40]:
### unque english and hindi words

In [41]:
all_eng_words=set()
for eng in lines.eword:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
all_hindi_words=set()
for hin in lines.hword:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [42]:
len(all_eng_words), len(all_hindi_words)

(4517, 5359)

In [43]:
###finding the maximum length of a sentence in english and hindi

In [44]:
lenght_list=[]
for l in lines.hword:
    lenght_list.append(len(l.split(' ')))
np.max(lenght_list)

17

In [45]:
lenght_list=[]
for l in lines.eword:
    lenght_list.append(len(l.split(' ')))
np.max(lenght_list)

7

In [46]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)

In [47]:
input_token_index = dict([(word, i) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i) for i, word in enumerate(target_words)])

In [48]:
encoder_input_data = np.zeros( (len(lines.eword), 7), dtype='float32')
decoder_input_data = np.zeros((len(lines.hword), 17),dtype='float32')
decoder_target_data = np.zeros((len(lines.hword), 17, num_decoder_tokens),dtype='float32')

In [49]:
for i, (input_text, target_text) in enumerate(zip(lines.eword, lines.hword)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

### Building keras encoder decoder model

In [50]:
embedding_size = 50

### encoder model

In [51]:
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

### decoder model

In [52]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

dex=  Embedding(num_decoder_tokens, embedding_size)

final_dex= dex(decoder_inputs)


decoder_lstm = LSTM(50, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(final_dex,initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [53]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,batch_size=32,epochs=27,validation_split=0.05)

Train on 28500 samples, validate on 1500 samples
Epoch 1/27
28500/28500 [==============================] - 675s 24ms/step - loss: 1.5468 - acc: 0.0723 - val_loss: 1.7391 - val_acc: 0.0811
Epoch 2/27
28500/28500 [==============================] - 668s 23ms/step - loss: 1.3027 - acc: 0.0978 - val_loss: 1.5694 - val_acc: 0.1178
Epoch 3/27
28500/28500 [==============================] - 670s 24ms/step - loss: 1.1728 - acc: 0.1240 - val_loss: 1.4630 - val_acc: 0.1410
Epoch 4/27
28500/28500 [==============================] - 656s 23ms/step - loss: 1.0894 - acc: 0.1391 - val_loss: 1.3936 - val_acc: 0.1512
Epoch 5/27
28500/28500 [==============================] - 675s 24ms/step - loss: 1.0266 - acc: 0.1504 - val_loss: 1.3355 - val_acc: 0.1635
Epoch 6/27
28500/28500 [==============================] - 678s 24ms/step - loss: 0.9759 - acc: 0.1593 - val_loss: 1.2935 - val_acc: 0.1696
Epoch 7/27
28500/28500 [==============================] - 585s 21ms/step - loss: 0.9347 - acc: 0.1672 - val_loss: 1.2

In [54]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, None, 50)          225850    
_________________________________________________________________
lstm_3 (LSTM)                [(None, 50), (None, 50),  20200     
Total params: 246,050
Trainable params: 246,050
Non-trainable params: 0
_________________________________________________________________


In [55]:
### creating sampling sequence

In [56]:
decoder_state_input_h = Input(shape=(50,))
decoder_state_input_c = Input(shape=(50,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [57]:
### function to generate the sequence

In [58]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 52):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [59]:
### testing on sample inputs

In [60]:
for seq_index in [300,200,400,500, 600, 700, 1, 235, 2000, 1000]:
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', lines.eword[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: 300    stop that
Name: eword, dtype: object
Decoded sentence:  उसे मत दो । _END
-
Input sentence: 200    its here
Name: eword, dtype: object
Decoded sentence:  यह यहाँ है _END
-
Input sentence: 400    you drive
Name: eword, dtype: object
Decoded sentence:  आप की ज़रूरत है। _END
-
Input sentence: 500    did we win
Name: eword, dtype: object
Decoded sentence:  क्या हम इसे हैं _END
-
Input sentence: 600    i cant go
Name: eword, dtype: object
Decoded sentence:  मैं नहीं जा सकता हूँ _END
-
Input sentence: 700    i want you
Name: eword, dtype: object
Decoded sentence:  मैं तुम पर भरोसा करता हूं । _END
-
Input sentence: 1    run
Name: eword, dtype: object
Decoded sentence:  _END
-
Input sentence: 235    lie still
Name: eword, dtype: object
Decoded sentence:  चलो भी बस _END
-
Input sentence: 2000    i got dizzy
Name: eword, dtype: object
Decoded sentence:  मुझे कुछ मैं एक आ रहा है । _END
-
Input sentence: 1000    drop it
Name: eword, dtype: object
Decoded sentence:  इसे ले ल